In [ ]:
!pip install transformers==4.9.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.6 MB 15.2 MB/s 
     |████████████████████████████████| 3.3 MB 62.3 MB/s 
     |████████████████████████████████| 880 kB 70.6 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=195c15743b3abb97c2a4fb20237156288674338f3ff90339e0eeea1ae1b096aa
  Stored in directory: /root/.cache/pip/wheels/82/ab/9b/c15899bf659ba74f623ac776e861cf2eb8608c1825ddec66a4
Successfully built sacremoses


In [ ]:
!pip install datasets==1.10.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 542 kB 14.6 MB/s 
     |████████████████████████████████| 212 kB 70.7 MB/s 
     |████████████████████████████████| 132 kB 70.3 MB/s 


In [ ]:
import traceback
import csv

import pandas as pd


def write_tsv_dataframe(filepath, dataframe):
    """
       `DataFrame` is stored as tsv file
        Raises IOError if file is not been able open
    """
    try:
        dataframe.to_csv(filepath, encoding='utf-8', sep='\t', index=False, header=True, quoting=csv.QUOTE_NONE)
    except IOError:
        traceback.print_exc()

In [ ]:
import pandas as pd


def combine_columns(df_arg, df_labels):
    """merges the two "DataFrames" on the "Argument ID" column."""
    return pd.merge(df_arg, df_labels, on='Argument ID')


def split_arguments(df_arg):
    """Splits `DataFrame` by column `Usage` into `train`-, `validation`-, and `test`-arguments"""
    train_arg = df_arg.loc[df_arg['Usage'] == 'train'].drop(['Usage'], axis=1).reset_index(drop=True)
    test_arg = df_arg.loc[df_arg['Usage'] == 'test'].drop(['Usage'], axis=1).reset_index(drop=True)
    valid_arg = df_arg.loc[df_arg['Usage'] == 'validation'].drop(['Usage'], axis=1).reset_index(drop=True)
    
    return train_arg,  test_arg, valid_arg


def create_dataframe_head(arg_ids, model_name):
    """
        merges the two "DataFrames" on the "Argument ID" column. creates a "DataFrame" that can have predictions added to it.
        Parameters
              Argument ids are as follows: list[str]
              The generated DataFrame model name's first column is str.
              The model name will appear in the second column of the DataFrame.
        Returns 
          A DataFrame has been prepared.
    """
    df_modl_head = pd.DataFrame(arg_ids, columns=['Argument ID'])
    df_modl_head['Method'] = [model_name] * len(arg_ids)

    return df_modl_head

In [ ]:
import traceback
import pandas as pd
import json


class MissingColumnError(AttributeError):
   """Error: An imported DataFrame does not include the required columns"""
    pass


def load_json_file(filepath):
    with open(filepath, 'r') as  json_file:
        return json.load(json_file)


def load_values_from_json(filepath):
    json_val = load_json_file(filepath)
    vals = { "1":set(), "2":set(), "3":set(), "4a":set(), "4b":set() }
    for v in json_val["vals"]:
        vals["1"].add(v["name"])
        vals["2"].add(v["level2"])
        for valLevel3 in v["level3"]:
            vals["3"].add(valLevel3)
        for valLevel4a in v["level4a"]:
            vals["4a"].add(valLevel4a)
        for valLevel4b in v["level4b"]:
            vals["4b"].add(valLevel4b)
    vals["1"] = sorted(vals["1"])
    vals["2"] = sorted(vals["2"])
    vals["3"] = sorted(vals["3"])
    vals["4a"] = sorted(vals["4a"])
    vals["4b"] = sorted(vals["4b"])
    return vals


def load_arguments_from_tsv(filepath, default_usage='test'):
    """
      "Reading parameters from a tsv file.
        Parameters
        —————
        str filepath
        The tsv file's location is default usage: str, optional
        the default value if the "Usage" column is empty
        Returns
        ———-\s pd.
        the DataFrame and include all arguments.
        Raises
        ———
        If the needed columns "Argument ID" or "Premise" are missing in the read data, a MissingColumnError will occur, and an IOError will occur if the file cannot be read "
    """
    try:
        d_frame = pd.read_csv(filepath, encoding='utf-8', sep='\t', header=0)
        if not {'Argument ID', 'Premise'}.issubset(set(d_frame.columns.values)):
            raise MissingColumnError('The argument "%s" file does not contain the minimum required columns [Argument ID, Premise].' % filepath)
        if 'Usage' not in d_frame.columns.values:
            d_frame['Usage'] = [default_usage] * len(d_frame)
        return d_frame
    except IOError:
        traceback.print_exc()
        raise


def load_labels_from_tsv(filepath, label_order):

    try:
        d_frame = pd.read_csv(filepath, encoding='utf-8', sep='\t', header=0)
        d_frame = dataframe[['Argument ID'] + label_order]
        return d_frame
    except IOError:
        traceback.print_exc()
        raise
    except KeyError:
        raise MissingColumnError('The file "%s" does not contain the required columns for its level.' % filepath)

In [ ]:
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 15.6 MB/s 


In [ ]:
!pip install numpy==1.19.3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 14.9 MB 4.3 MB/s 
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.6
    Uninstalling numpy-1.21.6:
      Successfully uninstalled numpy-1.21.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.9.2 requires numpy>=1.20, but you have numpy 1.19.3 which is incompatible.
jaxlib 0.3.25+cuda11.cudnn805 requires numpy>=1.20, but you have numpy 1.19.3 which is incompatible.
jax 0.3.25 requires numpy>=1.20, but you have numpy 1.19.3 which is incompatible.
cupy-cuda11x 11.0.0 requires numpy<1.26,>=1.20, but you have numpy 1.19.3 which is incompatible.
cmdstanpy 1.0.8 requires numpy>=1.21, but you have numpy 1.19.3 which is incompatible.


In [ ]:
import torch
from datasets import (Dataset, load_dataset, DatasetDict)
from transformers import (AutoTokenizer, AutoModelForSequenceClassification, PreTrainedModel, BertModel, BertForSequenceClassification, TrainingArguments, Trainer)
from sklearn.metrics import f1_score
# from transformers import MBartTokenizer
from transformers import RobertaTokenizer
from transformers import LongformerTokenizer
from transformers import DistilBertTokenizer, DistilBertTokenizerFast, DistilBertModel
import numpy as np


def accuracy_pred(y_prediction, y_true, thresh=0.5, sigmoid=True):
    """Calculate prediction accuracy"""
    y_prediction = torch.from_numpy(y_pred)
    y_true = torch.from_numpy(y_true)
    if sigmoid:
        y_prediction = y_prediction.sigmoid()

    return ((y_prediction > thresh) == y_true.bool()).float().mean().item()


def f1_score_each_label(y_pred, y_true, value_classes, thresh=0.5, sigmoid=True):
    """Calculate label-wise and average F1 scores"""
    y_pred = torch.from_numpy(y_pred)
    y_true = torch.from_numpy(y_true)
    if sigmoid:
        y_pred = y_pred.sigmoid()

    y_true = y_true.bool().numpy()
    y_pred = (y_pred > thresh).numpy()

    f1_score = {}
    for j, v in enumerate(value_classes):
        f1_score[v] = round(f1_score(y_true[:, j], y_pred[:, j], zero_division=0), 2)

    f1_score['avg-f1-score'] = round(np.mean(list(f1_score.values())), 2)

    return f1_score


def calc_metrics(eval_pred, value_classes):
    """MultiLabelTrainer's custom metric calculation function"""
    preds, labels_ = eval_pred
    f1scores = f1_score_per_label(preds, labels_, value_classes)
    return {'accuracy_pred': accuracy_pred(preds, labels_), 'f1-score': f1scores,
            'marco-avg-f1score': f1scores['avg-f1-score']}


class MultiLabelTrainer(Trainer):
   
    def calc_loss(self, model, inputs, return_outputs=False):
        """loss computation"""
        labels = inputs.pop("labels")
        out_put = model(**inputs)
        logits = out_put.logits
        loss_fct = torch.nn.BCEWithLogitsLoss()
        loss_ = loss_fct(logits.view(-1, self.model.config.num_labels),
                        labels.float().view(-1, self.model.config.num_labels))
        return (loss_, out_put) if return_outputs else loss_


def tokenize_and_encode(examples):
  
    return tokenizer(examples['Premise'], truncation=True)


def convert_to_dataset(train_df, test_df, labels):
    """
        creates a DatasetDict from a pandas DataFrame.
        
        """
    column_x = [i for i in (['Premise'] + labels) if i in train_df.columns.values]

    train_data = Dataset.from_dict((train_df[column_x]).to_dict('list'))
    test_data = Dataset.from_dict((test_df[column_x]).to_dict('list'))

    d_s = DatasetDict()
    d_s['train'] = train_data
    d_s['test'] = test_data
    d_s = d_s.map(lambda x: {"labels": [int(x[c]) for c in d_s['train'].column_names if
                                      c not in ['Argument ID', 'Conclusion', 'Stance', 'Premise', 'Part']]})

    columns = ds['train'].column_names
    columns.remove('labels')

    ds_encode = ds.map(tokenize_and_encode, batched=True, remove_columns=cols)

    columns.remove('Premise')

    return ds_encode, columns


tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")


# tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
# tokenizer = BertTokenizer.from_pretrained("prajjwal1/bert-tiny")
# tokenizer = BertTokenizer.from_pretrained("prajjwal1/bert-small")
# tokenizer = BertTokenizer.from_pretrained("prajjwal1/bert-medium")
# tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased") 
# tokenizer = RobertaTokenizer.from_pretrained("roberta-base") 
# tokenizer = LongformerTokenizer.from_pretrained("allenai/longformer-base-4096")

def load_model_from_data_dir(dir_model, num_labels):
    """Loads the Bert model from the directory and, if possible, turns it to a CUDA model."""
    model_loaded = AutoModelForSequenceClassification.from_pretrained(dir_model, num_labels=num_labels)
    if torch.cuda.is_available():
        return model_loaded.to('cuda')
    return model_loaded


def predict_bert_model(dataframe, dir_model, labels):
    """
        classifies each parameter using the Bert model saved in "dir model," providing a numpy nd-array containing the model's predictions.
        """
    ds, no_labels = convert_to_dataset(dataframe, dataframe, labels)
    num_label = len(labels)
    ds = ds.remove_columns(['labels'])

    batch_size = 8
    args = TrainingArguments(
        output_dir=dir_model,
        do_train=False,
        do_eval=False,
        do_predict=True,
        per_device_eval_batch_size=batch_size
    )

    model = load_model_from_data_dir(dir_model, num_label=num_label)

    multi_train = MultiLabelTrainer(
        model,
        args,
        tokenizer=tokenizer
    )

    predictions = 1 * (multi_train.predict(ds['train']).predictions > 0.5)

    return predictions


def train_bert_model(train_df, model_dir, labels, test_df=None, num_train_epochs=20):
    
    if test_df is None:
        test_df = train_df
    ds, labels = convert_to_dataset(train_df, test_df, labels)

    batch_size = 8

    args = TrainingArguments(
        output_dir=model_dir,
        evaluation_strategy="steps",
        learning_rate=2e-5,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        num_train_epochs=num_train_epochs,
        weight_decay=0.01,
        load_best_model_at_end=True,
        metric_for_best_model='marco-avg-f1score'
    )

    model = load_model_from_data_dir("bert-base-uncased", num_label=len(labels))

    multi_train = MultiLabelTrainer(
        model,
        args,
        train_data=ds["train"],
        eval_data=ds["test"],
        compute_metrics=lambda x: compute_metrics(x, labels),
        tokenizer=tokenizer
    )

    multi_trainer.train()

    model.save_pretrained(model_dir)

    if test_df is not None:
        return multi_trainer.evaluate()

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
import numpy as np
import pandas as pd


def predict_one_baseline(dataframe, labels):
    """
        classifies each dataframe argument as belonging to one of the labels.
        returns a DataFrame containing the model's predictions.
        """
    return pd.DataFrame(np.full((len(dataframe), len(labels)), 1, dtype=int), columns=labels)

In [ ]:

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import f1_score
from sklearn.svm import LinearSVC

import pandas as pd
import numpy as np

import json

# constant label values
vocab_label = 'vocabulary'
idf_label = 'idf'
intercept_label = 'intercept'
coef_label = 'coef'


class MyLinearSVC(LinearSVC):
    """
       a class for storing and loading a pretrained linear svm for predictions
    
    """

    def __init__(self, intercept, coeff):
        """
            creates every required attribute for the MyLinearSVC object.
            
        """
        LinearSVC.__init__(self, C=18, class_weight='balanced', max_iter=10000)
        self.intercept = intercept
        self.coeff = np.asarray(coeff)
        self.size = len(coeff)

    def predict(self, X):
        """
            Predict class labels for samples in X
        """
        input = np.transpose(X.todense())
        matrices = np.squeeze(np.asarray(self.__my_predict(input)))
        return np.vectorize(lambda x: 1 if x >= 0.5 else 0)(matrices)

    def __my_predict(self, input):
        """
            Predict class label probability for samples in input

        """
        res = input[0] * self.coeff[0]
        for i in range(1, self.size):
            res += input[i] * self.coeff[i]
        res += self.intercept
        return res


def predict_svm(dataframe, labels, vectorizer_file, model_file):
    """
       use the Support Vector Machines (SVMs) in the model file to classify each parameter in the dataframe.
        Returns the model's predictions are presented in a dataframe.
    """
    input_vector = dataframe['Premise']
    df_model_predictions = {}

    # load vectorizer
    with open(vectorizer_file, "r") as f:
        vectorizer_Json = json.load(f)

    vocab= vectorizer_Json[vocab_label]
    idf = np.asarray(vectorizer_Json[idf_label])

    vectorizer = TfidfVectorizer(vocab=vocab)
    # vectorizer = CountVectorizer(vocab=voca)
    # vectors = vectorizer.fit_transform(corpus)
    vectorizer.idf_ = idf

    with open(model_file, "r") as f:
        model_json = json.load(f)

    for label_names in labels:
        modelDict = model_json[label_names]
        svm = Pipeline([
            ('tfidf', vectorizer),
            ('clf', MyLinearSVC(intercept=modelDict[intercept_label], coef=modelDict[coef_label])),
        ])
        df_model_predict[label_names] = svm.predict(input_vector)

    return pd.DataFrame(df_model_predict, columns=labels)


def train_svm(train_df, labels, vectorizer_file, model_file, test_df=None):
    """
        SVMs are trained using the parameters in the train dataframe, and their training results are saved in the model file.
      f1-scores of the validation are returned if "test dataframe" is not None.
      
        """
    train_inputVector = train_df['Premise']
    if test_df is not None:
        valid_input_vector = test_df['Premise']
        f1_scores = {}

    # vectorizer = CountVectorizer(stop_words='english')
    vectorizer = TfidfVectorizer(stop_words='english')
    vectorizer.fit(train_inputVector)

    vectorizer_Json = {vocab_label: vectorizer.vocabulary_, idf_label: vectorizer.idf_.tolist()}

    with open(vectorizer_file, "w") as f:
        json.dump(vectorizer_json, f)

    # dictionary for storing model data
    model_Json = {}

    for label_name in labels:
        svm = Pipeline([
            ('tfidf', vectorizer),
            ('clf', OneVsRestClassifier(LinearSVC(C=18, class_weight='balanced', max_iter=20000), n_jobs=1)),
        ])
        svm.fit(train_input_vector, train_dataframe[label_name])

        coef = np.squeeze(np.asarray(svm.steps[1][1].estimators_[0].coef_)).tolist()
        intercept = svm.steps[1][1].estimators_[0].intercept_[0]
        model_Json[label_name] = {intercept_label: intercept, coef_label: coef}

        if test_dataframe is not None:
            valid_pred = svm.predict(valid_input_vector)
            f1_scores[label_name] = round(f1_score(test_dataframe[label_name], valid_pred, zero_division=0), 2)

    with open(model_file, "w") as f:
        json.dump(model_json, f)

    if test_df is not None:
        f1_scores['avg-f1-score'] = round(np.mean(list(f1_scores.values())), 2)
        return f1_scores

In [ ]:
import sys
import getopt
import os

# from components.setup import (load_values_from_json, load_arguments_from_tsv, load_labels_from_tsv,
#                                                 combine_columns, split_arguments)
# from components.models import (train_bert_model, train_svm)

help_str = '\nUsage:  training.py [OPTIONS]' \
              '\n' \
              '\nTrain the BERT model (and optional SVM) on the arguments' \
              '\n' \
              '\nOptions:' \
              '\n  -c, --classifier string  Select classifier: "b" for Bert, "s" for SVM, "bs" for both (default' \
              '\n                           "b")' \
              '\n  -d, --data-dir string    Directory with the argument files (default "/data/")' \
              '\n  -h, --help               Display help text' \
              '\n  -l, --levels string      Comma-separated list of taxonomy levels to train models for (default' \
              '\n                           "1,2,3,4a,4b")' \
              '\n  -m, --model-dir string   Directory for saving the trained models (default "/models/")' \
              '\n  -v, --validate           Request evaluation after training'


def main():
    # default values
    curr_dir = os.getcwd()
    run_bert = True
    run_svm = False
    data_dir = '/content/data_dir'
    levels = ["2"]
    model_dir = '/content/models/'
    validate = True

    svm_dir = os.path.join(model_dir, 'svm')

    # Check data directory
    if not os.path.isdir(data_dir):
        print('The data directory "%s" does not exist' % data_dir)
        sys.exit(2)

    # Check model directory
    if os.path.isfile(model_dir):
        print('The <model-dir> "%s" points to an existing file' % model_dir)
        sys.exit(2)
    if os.path.isdir(model_dir) and len(os.listdir(model_dir)) > 0:
        print('The <model-dir> "%s" already exists and contains files' % model_dir)
        deci = input('Do you want to proceed? [yes/no]\n').lower()
        if deci != 'y':
            sys.exit(-1)
    if not os.path.exists(model_dir):
        os.makedirs(model_dir)

    if run_svm and not os.path.isdir(SVM_dir):
        if os.path.exists(SVM_dir):
            print('Not able to create svm directory at "%s"' % SVM_dir)
        else:
            os.mkdir(SVM_dir)

    arg_filepath = os.path.join(data_dir, 'arguments.tsv')
    value_json_filepath = os.path.join(data_dir, 'values.json')

    if not os.path.isfile(arg_filepath):
        print('The required file "arguments.tsv" is not there in the data directory')
        sys.exit(2)
    if not os.path.isfile(value_json_filepath):
        print('The required file "values.json" is not there in the data directory')
        sys.exit(2)

    # load arguments
    df_arg = load_arguments_from_tsv(arg_filepath, default_usage='train')
    if len(df_arg) < 1:
        print('Ard is not there in the  file "%s"' % arg_filepath)
        sys.exit(2)

    val = load_values_from_json(value_json_filepath)
    number_levels = len(levels)

    # check levels
    for i in range(number_levels):
        if levels[i] not in val:
            print('Missing attribute "{}" in value.json'.format(levels[i]))
            sys.exit(2)

    # format dataset
    df_train_all = []
    df_valid_all = []
    for i in range(number_levels):
        label_filepath = os.path.join(data_dir, 'labels-level{}.tsv'.format(levels[i]))
        if not os.path.isfile(label_filepath):
            print('The required file "labels-level{}.tsv" is not present in the data directory'.format(levels[i]))
            sys.exit(2)
        # read labels from .tsv file
        df_labels = load_labels_from_tsv(label_filepath, values[levels[i]])
        # join arguments and labels
        df_full_level = combine_columns(df_arguments, df_labels)
        # split dataframe by usage
        train_args, valid_args, _ = split_arguments(df_full_level)
        df_train_all.append(train_args)
        df_valid_all.append(valid_args)

    if len(df_train_all[0]) < 1:
        print('There are no arguments listed for training.')
        sys.exit()

    if validate and len(df_valid_all[0]) < 1:
        print('There are no arguments listed for validation. Proceeding without validation.')
        validate = False

    # train bert model
    if run_bert:
        for i in range(number_levels):
            print("===> Bert: Training Level %s..." % levels[i])
            if validate:
                bert_model_evaluation = train_bert_model(df_train_all[i],
                                                         os.path.join(model_dir, 'bert_train_level{}'.format(levels[i])),
                                                         values[levels[i]], test_dataframe=df_valid_all[i])
                print("F1-Scores for Level %s:" % levels[i])
                print(bert_model_evaluation['eval_f1-score'])
            else:
                train_bert_model(df_train_all[i], os.path.join(model_dir, 'bert_train_level{}'.format(levels[i])),
                                 values[levels[i]])

    if run_svm:
        for i in range(number_levels):
            print("===> SVM: Training Level %s..." % levels[i])
            if validate:
                svm_f1scores = train_svm(df_train_all[i], values[levels[i]],
                                          os.path.join(model_dir, 'svm/svm_train_level{}_vectorizer.json'.format(levels[i])),
                                          os.path.join(model_dir, 'svm/svm_train_level{}_models.json'.format(levels[i])),
                                          test_dataframe=df_valid_all[i])
                print("F1-Scores for Level %s:" % levels[i])
                print(svm_f1scores)
            else:
                train_svm(df_train_all[i], values[levels[i]],
                          os.path.join(model_dir, 'svm/svm_train_level{}_vectorizer.json'.format(levels[i])),
                          os.path.join(model_dir, 'svm/svm_train_level{}_models.json'.format(levels[i])))


if __name__ == '__main__':
    main()

===> Bert: Training Level 2...


  0%|          | 0/4240 [00:00<?, ?ex/s]

  0%|          | 0/277 [00:00<?, ?ex/s]

  0%|          | 0/5 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

...

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Step,Training Loss,Validation Loss,Accuracy Thresh,F1-score,Marco-avg-f1score
500,0.417000,0.363368,0.856679,"{'Achievement': 0.42, 'Benevolence: caring': 0.04, 'Benevolence: dependability': 0.0, 'Conformity: interpersonal': 0.0, 'Conformity: rules': 0.0, 'Face': 0.0, 'Hedonism': 0.0, 'Humility': 0.0, 'Power: dominance': 0.0, 'Power: resources': 0.0, 'Security: personal': 0.31, 'Security: societal': 0.69, 'Self-direction: action': 0.45, 'Self-direction: thought': 0.52, 'Stimulation': 0.0, 'Tradition': 0.61, 'Universalism: concern': 0.63, 'Universalism: nature': 0.0, 'Universalism: objectivity': 0.0, 'Universalism: tolerance': 0.0, 'avg-f1-score': 0.18}",0.180000
1000,0.343500,0.344992,0.863538,"{'Achievement': 0.4, 'Benevolence: caring': 0.19, 'Benevolence: dependability': 0.0, 'Conformity: interpersonal': 0.0, 'Conformity: rules': 0.25, 'Face': 0.0, 'Hedonism': 0.0, 'Humility': 0.0, 'Power: dominance': 0.0, 'Power: resources': 0.0, 'Security: personal': 0.55, 'Security: societal': 0.78, 'Self-direction: action': 0.5, 'Self-direction: thought': 0.62, 'Stimulation': 0.0, 'Tradition': 0.55, 'Universalism: concern': 0.73, 'Universalism: nature': 0.0, 'Universalism: objectivity': 0.0, 'Universalism: tolerance': 0.06, 'avg-f1-score': 0.23}",0.230000
1500,0.300600,0.332273,0.864079,"{'Achievement': 0.64, 'Benevolence: caring': 0.26, 'Benevolence: dependability': 0.0, 'Conformity: interpersonal': 0.0, 'Conformity: rules': 0.44, 'Face': 0.0, 'Hedonism': 0.0, 'Humility': 0.0, 'Power: dominance': 0.0, 'Power: resources': 0.44, 'Security: personal': 0.5, 'Security: societal': 0.73, 'Self-direction: action': 0.55, 'Self-direction: thought': 0.67, 'Stimulation': 0.0, 'Tradition': 0.29, 'Universalism: concern': 0.69, 'Universalism: nature': 0.0, 'Universalism: objectivity': 0.21, 'Universalism: tolerance': 0.06, 'avg-f1-score': 0.27}",0.270000
2000,0.271400,0.330668,0.865162,"{'Achievement': 0.66, 'Benevolence: caring': 0.3, 'Benevolence: dependability': 0.0, 'Conformity: interpersonal': 0.0, 'Conformity: rules': 0.41, 'Face': 0.0, 'Hedonism': 0.0, 'Humility': 0.0, 'Power: dominance': 0.0, 'Power: resources': 0.54, 'Security: personal': 0.5, 'Security: societal': 0.77, 'Self-direction: action': 0.57, 'Self-direction: thought': 0.65, 'Stimulation': 0.0, 'Tradition': 0.53, 'Universalism: concern': 0.7, 'Universalism: nature': 0.0, 'Universalism: objectivity': 0.22, 'Universalism: tolerance': 0.28, 'avg-f1-score': 0.31}",0.310000
2500,0.234900,0.330648,0.867509,"{'Achievement': 0.66, 'Benevolence: caring': 0.35, 'Benevolence: dependability': 0.0, 'Conformity: interpersonal': 0.0, 'Conformity: rules': 0.43, 'Face': 0.0, 'Hedonism': 0.18, 'Humility': 0.0, 'Power: dominance': 0.0, 'Power: resources': 0.55, 'Security: personal': 0.5, 'Security: societal': 0.76, 'Self-direction: action': 0.62, 'Self-direction: thought': 0.63, 'Stimulation': 0.17, 'Tradition': 0.68, 'Universalism: concern': 0.71, 'Universalism: nature': 0.25, 'Universalism: objectivity': 0.17, 'Universalism: tolerance': 0.29, 'avg-f1-score': 0.35}",0.350000
3000,0.208500,0.333413,0.867690,"{'Achievement': 0.66, 'Benevolence: caring': 0.32, 'Benevolence: dependability': 0.0, 'Conformity: interpersonal': 0.0, 'Conformity: rules': 0.35, 'Face': 0.0, 'Hedonism': 0.12, 'Humility': 0.0, 'Power: dominance': 0.12, 'Power: resources': 0.56, 'Security: personal': 0.53, 'Security: societal': 0.75, 'Self-direction: action': 0.6, 'Self-direction: thought': 0.65, 'Stimulation': 0.14, 'Tradition': 0.72, 'Universalism: concern': 0.7, 'Universalism: nature': 0.0, 'Universalism: objectivity': 0.22, 'Universalism: tolerance': 0.35, 'avg-f1-score': 0.34}",0.340000
3500,0.179800,0.343041,0.868773,"{'Achievement': 0.69, 'Benevolence: caring': 0.28, 'Benevolence: dependability': 0.0, 'Conformity: interpersonal': 0.2, 'Conformity: rules': 0.42, 'Face': 0.0, 'Hedonism': 0.13, 'Humility': 0.0, 'Power: dominance': 0.12, 'Power: resources': 0.52, 'Security: personal': 0.51, 'Security: societal': 0.79, 'Self-di

***** Running Evaluation *****
  Num examples = 277
  Batch size = 8
Trainer is attempting to log a value of "{'Achievement': 0.42, 'Benevolence: caring': 0.04, 'Benevolence: dependability': 0.0, 'Conformity: interpersonal': 0.0, 'Conformity: rules': 0.0, 'Face': 0.0, 'Hedonism': 0.0, 'Humility': 0.0, 'Power: dominance': 0.0, 'Power: resources': 0.0, 'Security: personal': 0.31, 'Security: societal': 0.69, 'Self-direction: action': 0.45, 'Self-direction: thought': 0.52, 'Stimulation': 0.0, 'Tradition': 0.61, 'Universalism: concern': 0.63, 'Universalism: nature': 0.0, 'Universalism: objectivity': 0.0, 'Universalism: tolerance': 0.0, 'avg-f1-score': 0.18}" of type <class 'dict'> for key "eval/f1-score" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Saving model checkpoint to /content/models/bert_train_level2/checkpoint-500
Configuration saved in /content/models/bert_train_level2/checkpoint-500/config.json
Model weights saved in

Trainer is attempting to log a value of "{'Achievement': 0.67, 'Benevolence: caring': 0.31, 'Benevolence: dependability': 0.0, 'Conformity: interpersonal': 0.2, 'Conformity: rules': 0.42, 'Face': 0.0, 'Hedonism': 0.29, 'Humility': 0.0, 'Power: dominance': 0.19, 'Power: resources': 0.5, 'Security: personal': 0.56, 'Security: societal': 0.79, 'Self-direction: action': 0.6, 'Self-direction: thought': 0.67, 'Stimulation': 0.23, 'Tradition': 0.69, 'Universalism: concern': 0.74, 'Universalism: nature': 0.0, 'Universalism: objectivity': 0.27, 'Universalism: tolerance': 0.3, 'avg-f1-score': 0.37}" of type <class 'dict'> for key "eval/f1-score" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


F1-Scores for Level 2:
{'Achievement': 0.67, 'Benevolence: caring': 0.31, 'Benevolence: dependability': 0.0, 'Conformity: interpersonal': 0.2, 'Conformity: rules': 0.42, 'Face': 0.0, 'Hedonism': 0.29, 'Humility': 0.0, 'Power: dominance': 0.19, 'Power: resources': 0.5, 'Security: personal': 0.56, 'Security: societal': 0.79, 'Self-direction: action': 0.6, 'Self-direction: thought': 0.67, 'Stimulation': 0.23, 'Tradition': 0.69, 'Universalism: concern': 0.74, 'Universalism: nature': 0.0, 'Universalism: objectivity': 0.27, 'Universalism: tolerance': 0.3, 'avg-f1-score': 0.37}


In [ ]:
import sys
import getopt
import os
import pandas as pd

# from components.setup import (load_values_from_json, load_arguments_from_tsv, split_arguments,
#                               write_tsv_dataframe, create_dataframe_head)
# from components.models import (predict_bert_model, predict_one_baseline, predict_svm)

help_str = '\nUsage:  predict.py [OPTIONS]' \
              '\n' \
              '\nRequest prediction of the BERT model (and optional SVM / 1-Baseline) for all test arguments' \
              '\n' \
              '\nOptions:' \
              '\n  -c, --classifier string  Select classifier: "b" for Bert, "s" for SVM, "o" for 1-Baseline,' \
              '\n                           or combination like "so" (default "b")' \
              '\n  -d, --data-dir string    Directory with the argument files (default "/data/")' \
              '\n  -h, --help               Display help text' \
              '\n  -l, --levels string      Comma-separated list of taxonomy levels to train models for (default' \
              '\n                           "1,2,3,4a,4b")' \
              '\n  -m, --model-dir string   Directory for saving the trained models (default "/models/")' \
              '\n  -o, --output-dir string  Directory to write the "predictions.tsv" into (default "/output/")'


def main():
   
    curr_dir = os.getcwd()
    run_bert = True
    run_svm = False
    run_one_baseline = False
    data_dir = '/content/data_dir'
    levels = ["2"]
    model_dir = '/content/models/'
    output_dir = '/content/output/'

    
    if not os.path.isdir(data_dir):
        print('The specified data directory "%s" does not exist' % data_dir)
        sys.exit(2)

    arg_filepath = os.path.join(data_dir, 'arguments.tsv')
    values_filepath = os.path.join(data_dir, 'values.json')

    if not os.path.isfile(arg_filepath):
        print('The required file "arguments.tsv" is not present in the data directory')
        sys.exit(2)
    if not os.path.isfile(values_filepath):
        print('The required file "values.json" is not present in the data directory')
        sys.exit(2)

    # load arguments
    df_args = load_arguments_from_tsv(arg_filepath)
    if len(df_args) < 1:
        print('There are no arguments in file "%s"' % arg_filepath)
        sys.exit(2)

    val = load_values_from_json(values_filepath)
    num_levels = len(levels)

    # check levels
    for i in range(num_levels):
        if levels[i] not in val:
            print('Missing attribute "{}" in value.json'.format(levels[i]))
            sys.exit(2)


    if not os.path.isdir(model_dir):
        print('The specified <model-dir> "%s" does not exist' % model_dir)
        sys.exit(2)

    for j in range(num_levels):
        if run_bert and not os.path.exists(os.path.join(model_dir, 'bert_train_level{}'.format(levels[j]))):
            print('Missing saved Bert model for level "{}"'.format(levels[j]))
            sys.exit(2)
        if run_svm and (
                not os.path.exists(os.path.join(model_dir, 'svm/svm_train_level{}_vectorizer.json'.format(levels[j])))
                and not os.path.exists(os.path.join(model_dir, 'svm/svm_train_level{}_models.json'.format(levels[j])))):
            print('Missing saved SVM models for level "{}"'.format(levels[j]))
            sys.exit(2)

    # dataset
    _, _, df_test = split_arguments(df_arguments)

    if len(df_test) < 1:
        print('There are no arguments listed for prediction.')
        sys.exit()

    #  Bert model prediction
    if run_bert:
        df_bert_mod = create_dataframe_head(df_test['Argument ID'], model_name='Bert')
        for j in range(num_levels):
            print("===> Bert: Predicting Level %s..." % levels[j])
            results = predict_bert_model(df_test, os.path.join(model_dir, 'bert_train_level{}'.format(levels[j])),
                                        values[levels[j]])
            df_bert_mod = pd.concat([df_bert_mod, pd.DataFrame(results, columns=values[levels[j]])], axis=1)
        df_prediction = df_bert_mod

    # SVM prediction
    if run_svm:
        df_svm_mod = create_dataframe_head(df_test['Argument ID'], model_name='SVM')
        for j in range(num_levels):
            print("===> SVM: Predicting Level %s..." % levels[j])
            results = predict_svm(df_test, values[levels[j]],
                                 os.path.join(model_dir, 'svm/svm_train_level{}_vectorizer.json'.format(levels[j])),
                                 os.path.join(model_dir, 'svm/svm_train_level{}_models.json'.format(levels[j])))
            df_svm_mod = pd.concat([df_svm_mod, results], axis=1)

        if not run_bert:
            df_prediction = df_svm_mod
        else:
            df_prediction = pd.concat([df_prediction, df_svm_mod])

    #1-Baseline predivtion
    if run_one_baseline:
        df_one_baseline_mod = create_dataframe_head(df_test['Argument ID'], model_name='1-Baseline')
        for k in range(num_levels):
            print("===> 1-Baseline: Predicting Level %s..." % levels[k])
            results = predict_one_baseline(df_test, values[levels[k]])
            df_one_baseline_mod = pd.concat([df_one_baseline_mod, results], axis=1)

        if not run_bert and not run_svm:
            df_prediction = df_one_baseline_mod
        else:
            df_prediction = pd.concat([df_prediction, df_one_baseline_mod])

    # write predictions
    print("===> Writing predictions...")
    write_tsv_dataframe(os.path.join(output_dir, 'predictions_bert_2.tsv'), df_prediction)


if __name__ == '__main__':
    main()

===> Bert: Predicting Level 2...


  0%|          | 0/753 [00:00<?, ?ex/s]

  0%|          | 0/753 [00:00<?, ?ex/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
loading configuration file /content/models/bert_train_level2/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "

===> Writing predictions...
